In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
         
zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip').extractall()
zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip').extractall()

In [ ]:
from torch.utils import data
from torchvision import datasets, transforms
import torch
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import math
import datetime

import numpy as np

import time
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

!pip install timm
import timm

In [ ]:
EPOCHS = 10
LEARNING_RATE = 0.01
BATCH_SIZE = 64
VALID_SPLIT = 0.1
lr_step_size = 8
print_iter = 10

In [ ]:
def pil_loader(path, img_size=224):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    try:
        with open(path, "rb") as f:
            img = Image.open(f)
            return img.convert("RGB")
    except FileNotFoundError as e:
        raise FileNotFoundError(e)


def get_transform(random_crop=True):
    normalize = transforms.Normalize(
        mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
        std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
    transform = []
    transform.append(transforms.Resize(256))
    if random_crop:
        transform.append(transforms.RandomHorizontalFlip())
        transform.append(transforms.ColorJitter(brightness=0.1, saturation=0.1))
        # transform.append(transforms.RandomPerspective())
        transform.append(transforms.RandomResizedCrop(224))
    else:
        transform.append(transforms.CenterCrop(224))
    transform.append(transforms.ToTensor())
    transform.append(normalize)
    return transforms.Compose(transform)

In [ ]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2


def cutmix(input, target):
    input = input.clone().detach()
    target = target.clone().detach()
    beta = 1.0
    lam = np.random.beta(beta, beta)
    rand_index = torch.randperm(input.size()[0]).cuda()
    target_b = target[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(input.size(), lam)
    input[:, :, bbx1:bbx2, bby1:bby2] = input[rand_index, :, bbx1:bbx2, bby1:bby2]

    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (input.size()[-1] * input.size()[-2]))
    return target_b, lam

In [ ]:
class TestDataset(data.Dataset):
    def __init__(self, root='./test'):

        self.root = root
        self.samples = [[file_name, file_name.split('.')[0]] for file_name in os.listdir(self.root)]
        self.transform = get_transform(random_crop=False)

    def __getitem__(self, index):
        '''
        Here, our problem supposes maximum 3 hierarchy
        '''
        path, idx = self.samples[index]
        path = os.path.join(self.root, path)
        sample = self.transform(pil_loader(path=path))

        return torch.LongTensor([int(idx)]), sample

    def __len__(self):
        return len(self.samples)

In [ ]:
class CustomDataset(data.Dataset):
    def __init__(self, is_train=True, root='./train', split=1.0):

        self.root = root
        self.is_train = is_train
                        
        file_names = os.listdir(self.root)
                        
        test_size = 1.0 - split
        if test_size > 0:
            train_files, valid_files = train_test_split(file_names, test_size= split, random_state=97)
        else:
            train_files = file_names
            valid_files = []

        if is_train:
            random_crop = True
            dataset_files = train_files
        else:
            random_crop = False
            dataset_files = valid_files

        self.samples = np.array([[file_name, int(file_name.split('.')[0]=='dog'), int(file_name.split('.')[1])]
                        for file_name in dataset_files])
        target_names = np.unique(self.samples[:, 1])
        self.class_weights = torch.Tensor(compute_class_weight(class_weight='balanced', 
                                                                   classes=target_names,
                                                                   y=self.samples[:, 1]))
        self.transform = get_transform(random_crop=random_crop)

    def __getitem__(self, index):
        path, target, idx = self.samples[index]
        path = os.path.join(self.root, path)
        sample = self.transform(pil_loader(path=path))
        
        idx, target = int(idx), int(target)

        if self.is_train:
            return torch.tensor([idx]), sample, torch.tensor(target), self.class_weights
        return torch.tensor([idx]), sample, torch.tensor(target)

    def __len__(self):
        return len(self.samples)


In [ ]:

def data_loader(is_train=True, batch_size=16, split=1.0, Test=True):
    if Test:
        dataset = TestDataset()
    else:
        dataset = CustomDataset(is_train=is_train, split=split)
    return data.DataLoader(dataset=dataset,
                           batch_size=batch_size,
                           shuffle=is_train)

In [ ]:
class DnnClassifier(nn.Module):
    def __init__(self, in_features=1536):
        super(DnnClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=in_features, out_features=512)
        self.bn1 = nn.BatchNorm1d(512)
        
        self.fc2 = nn.Linear(in_features=512, out_features=128)
        self.bn2 = nn.BatchNorm1d(128)

        self.fc3 = nn.Linear(in_features=128, out_features=2)

    def forward(self, x):
        x = F.dropout(F.silu(self.bn1(self.fc1(x))))
        x = F.dropout(F.silu(self.bn2(self.fc2(x))))
        x = self.fc3(x)
        x = torch.sigmoid(x)

        return x

In [ ]:
class MySuperUltraUniverseFCN(nn.Module):
    def __init__(self):
        super(MySuperUltraUniverseFCN, self).__init__()
        self.out_features = 16
        self.conv1 = nn.Conv2d(3, self.out_features, kernel_size=224)  # 4 * 16
        self.bn1 = nn.BatchNorm2d(self.out_features)

    def forward(self, out):
        out = F.silu(self.bn1(self.conv1(out)))

        return out

In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self):
        super(EnsembleModel, self).__init__()
        self.out_features = 0
        self.modelA = timm.create_model('tf_efficientnetv2_l_in21ft1k', pretrained=True)
        self.out_features += self.modelA.classifier.in_features
        self.modelB = timm.create_model('rexnet_200', pretrained=True)
        self.out_features += self.modelB.head.fc.in_features
        self.modelC = timm.create_model('xception71', pretrained=True)
        self.out_features += self.modelC.head.fc.in_features
        self.modelA.classifier = nn.Identity()
        self.modelB.head.fc = nn.Identity()
        self.modelC.head.fc = nn.Identity()
        self.modelFCN = MySuperUltraUniverseFCN()
        
        self.out_features += self.modelFCN.out_features
        #self.classifier = DnnClassifier(in_features=self.out_features + self.modelFCN.out_features)
        
        for param in self.modelC.parameters():
            param.requires_grad = False
            
        for param in self.modelB.parameters():
            param.requires_grad = False
        
    def forward(self, out):
        out1 = self.modelA(out.clone())
        out1 = out1.view(out1.size(0), -1)

        out2 = self.modelB(out.clone())
        out2 = out2.view(out2.size(0), -1)

        out3 = self.modelC(out.clone())
        out3 = out3.view(out3.size(0), -1)

        out4 = self.modelFCN(out)
        out4 = out4.view(out4.size(0), -1)

        out = torch.cat((out1, out2, out3, out4), dim=1)
        #out = self.classifier(out)
        return out

In [ ]:
def reputation(pred, target):
    cnt = 0
    for pred_, target_ in zip(pred, target):
        if torch.argmax(pred_) == target_:
            cnt += 1
            
    return cnt

In [ ]:
model = EnsembleModel()
#for param in model.parameters():
#    param.requires_grad = False

classifier = DnnClassifier(in_features=model.out_features)

cuda = torch.cuda.is_available()
print('can use cuda', cuda)

if cuda:
    model = model.cuda()
    classifier = classifier.cuda()

In [ ]:
model_params = [param for param in model.parameters() if param.requires_grad]
classifier_params = [param for param in classifier.parameters() if param.requires_grad]

In [ ]:
optimizer = torch.optim.SGD(model_params + classifier_params, lr=LEARNING_RATE, momentum=0.9, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=lr_step_size, gamma=0.1)    

In [ ]:
train_loader = data_loader(is_train=True, split=VALID_SPLIT, batch_size=BATCH_SIZE, Test=False)
valid_loader = data_loader(is_train=False, split=VALID_SPLIT, batch_size=BATCH_SIZE, Test=False)

In [ ]:
import gc
gc.collect()

In [ ]:
global_iter = 0
for epoch in range(EPOCHS):
    
    # training
    model.train()
    train_loss = torch.tensor(0.)
    class_weights = [0.5, 0.5]
    num_trained_data = 0
    for iter_, data_ in enumerate(train_loader):
        global_iter += iter_
        
        _, x, label, class_weights = data_
        
        num_trained_data += label.shape[0]
        loss_fn = nn.CrossEntropyLoss(weight=class_weights[0])
        if cuda:
            x, label = x.cuda(), label.cuda()
            loss_fn = loss_fn.cuda()
            
        pred = model(x)
        pred = classifier(pred)
        
    
        if epoch >= 100:
            label_b, lam = cutmix(x, label)
            train_loss = loss_fn(pred, label) * lam + loss_fn(pred, label_b) * (1. - lam)
        else:
            train_loss = loss_fn(pred, label)
            
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if (iter_ + 1) % print_iter == 0:
            print('[',epoch+1,'/',EPOCHS,'] [',num_trained_data,'/',len(train_loader.dataset),'] train loss: ',float(train_loss))
        
    scheduler.step()
    
    Accuracy = 0.
    valid_size = len(valid_loader.dataset)
    valid_loss = torch.tensor(0.)
    if valid_size:
        model.eval()
        
        corrects = 0
        total_score = 0
        
        
        with torch.no_grad():
            for data_ in valid_loader:
                _, x, label, = data_
                loss_fn = nn.CrossEntropyLoss(weight=class_weights[0])
                
                if cuda:
                    x, label = x.cuda(), label.cuda()
                    loss_fn = loss_fn.cuda()
                    valid_loss = valid_loss.cuda()
                    
                pred = model(x)
                pred = classifier(pred)
                
                valid_loss = loss_fn(pred, label)
                
                corrects += reputation(pred, label)
        
        Accuracy = 100 * corrects / valid_size
    
    print('valid loss:', float(valid_loss))
    print('valid accuracy:', Accuracy)
                    

In [ ]:
test_loader = data_loader(is_train=False, split=VALID_SPLIT, batch_size=BATCH_SIZE, Test=True)

idx = np.array([])
preds = np.array([])
for data_ in test_loader:
    i, X_test = data_
    if cuda:
        X_test = X_test.cuda()
                    
    pred = model(X_test)
    pred = classifier(pred)
    
    preds = np.append(preds, torch.argmax(pred, dim=1).cpu().numpy())
    idx = np.append(idx, i.numpy())

d = {'id': idx, 'label':preds}
df = pd.DataFrame(data=d)
df = df.sort_values(by='id', ignore_index=True)
df.to_csv('./submission.csv', index=False)
    

In [ ]:
df.astype('Int32').to_csv('./submission.csv', index=False)